<a href="https://colab.research.google.com/github/GrigoryBartosh/dul_2021/blob/main/Homework/hw12/Homework12_semisupervised.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
!if [ -d dul_2021 ]; then rm -Rf dul_2021; fi
!git clone https://github.com/GrigoryBartosh/dul_2021
!pip install ./dul_2021

Cloning into 'dul_2021'...
remote: Enumerating objects: 384, done.
remote: Counting objects: 100% (221/221), done.
remote: Compressing objects: 100% (147/147), done.
remote: Total 384 (delta 124), reused 100 (delta 67), pack-reused 163
Receiving objects: 100% (384/384), 55.90 MiB | 2.75 MiB/s, done.
Resolving deltas: 100% (181/181), done.
Processing ./dul_2021
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Created wheel for dul-2021: filename=dul_2021-0.1.0-py3-none-any.whl size=27642 sha256=73ee2c5e1cfa079c750fd48655feedb3f5e97f94a54d0b059d893a5f98c6739c
  Stored in directory: /private/var/folders/zg/dd_4mghs167bb5hrrgz9w4y40000gn/T/pip-eph

In [31]:
from dul_2021.utils.hw12_utils import *

from torch import nn, optim
from torch.nn import functional as F
from torch.utils import data

from typing import TypeVar

import numpy as np

from tqdm.auto import tqdm


DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
TorchModel = TypeVar("TorchModel", bound=nn.Module)

# Question 1. VAT

Here we will implement [VAT](https://arxiv.org/pdf/1704.03976.pdf).

* Train labeled data with standatd cross-entropy loss

* Use vat regularization for both unlabeled and labeled data

* You can use architecture from practice

* Dataset comes as pairs `x, y`. `x` is an image from CIFAR10. `y` is a label from `[0, 10]` if datapoint is labeled and `-1` otherwise.

**Hyperparameters**

* ξ= 10 
* lr = 5e-4
* num_epochs = 15



**You will provide the following deliverables**


1. Over the course of training, record loss ber batch.
2. After each epoch calculate accuracy on test data.

In [32]:
class Net(nn.Module):
    def __init__(self, out_dim=128, hid_dim_full=128):
        super(Net, self).__init__()

        self.conv1 = nn.Conv2d(3, 16, 5, padding=2)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1, stride=2)
        self.conv3 = nn.Conv2d(16, 32, 5, padding=2)
        self.conv4 = nn.Conv2d(32, 32, 3, padding=1, stride=2)
        self.conv5 = nn.Conv2d(32, 32, 1)
        self.conv6 = nn.Conv2d(32, 4, 1)

        self.conv_to_fc = 8 * 8 * 4
        self.fc1 = nn.Linear(self.conv_to_fc, hid_dim_full)
        self.fc2 = nn.Linear(hid_dim_full, out_dim)

    def forward(self, x):

        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = F.relu(self.conv5(x))
        x = F.relu(self.conv6(x))

        x = x.view(-1, self.conv_to_fc)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))

        return x.log_softmax(-1)

In [33]:
class VATReg(nn.Module):
    def __init__(self, xi, eps=1.0):
        super().__init__()
        self._xi = xi
        self._eps = eps

    def forward(self, model: TorchModel, x: torch.Tensor):
        with torch.no_grad():
            p1 = model(x).exp()

        d = torch.randn_like(x)
        d /= d.view(d.size(0), -1).norm(dim=-1)[:, None, None, None].requires_grad_()

        model.eval()

        log_p2 = model(x + self._xi * d)
        d = F.kl_div(log_p2, p1, reduction='batchmean')
        d.backward()

        g = d.grad
        g /= g.view(g.size(0), -1).norm(dim=-1)[:, None, None, None]

        model.zero_grad()

        log_p3 = model(x + g * self._eps)
        return F.kl_div(log_p3, p1, reduction='batchmean')


class VAT(nn.Module):
    def __init__(self, out_dim=10, hidden_dim=128, xi=10.0, eps=1.0):
        super().__init__()
        self.net = Net(out_dim, hidden_dim)

        self.reg = VATReg(xi, eps)

    @property
    def device(self):
        return next(self.parameters()).device

    def fit(self, train_dl, test_dl, n_epochs, lr=1e-3):
        optimizer = optim.Adam(self.parameters(), lr=lr)

        losses, accuracies = [], []
        metric_dict = {}

        with tqdm(total=(len(train_dl) + len(test_dl)) * n_epochs) as pbar:
            for _ in range(n_epochs):
                self.train()
                for batch in train_dl:
                    x, y = batch
                    x, y = x.to(self.device), y.to(self.device)

                    mask = y > -1
                    loss = F.nll_loss(self.net(x[mask]), y[mask]) + self.reg(self.net, x)

                    loss.backward()
                    optimizer.step()
                    optimizer.zero_grad()

                    metric_dict['loss'] = loss.item()
                    losses.append(metric_dict['loss'])

                    pbar.update()
                    pbar.set_postfix(metric_dict)

                correct, total = 0, 0
                self.eval()
                with torch.no_grad():
                    for batch in test_dl:
                        x, y = batch
                        x, y = x.to(self.device), y.to(self.device)

                        pred = self.net(x[mask]).argmax(-1)

                        correct += (pred == y).sum().item()
                        total += x.size(0)

                        pbar.update()

                    metric_dict['acc'] = correct / total
                    accuracies.append(metric_dict['acc'])

        return np.array(losses), np.array(accuracies)


In [35]:
def q1(train_data, test_data):
    """
    train_data: An (n_train, 3, 32, 32) torchvision dataset of CIFAR10 images with values from -1 to 1

    Returns
    - a (# of training iterations, ) numpy array  losses on each iteration
    - a (# of training epochs, ) numpy array accuracies on each epoch
    """

    train_dl = data.DataLoader(train_data, batch_size=256, drop_last=True, shuffle=True)
    test_dl = data.DataLoader(test_data, batch_size=256)

    vat = VAT().to(DEVICE)

    losses, accuracies = vat.fit(train_dl, test_dl)

    return losses, accuracies


In [ ]:
q12_results(q1)

  0%|          | 0/170498071 [00:00<?, ?it/s]

# Question 2. FixMatch

Here we will implement [FixMatch](https://arxiv.org/abs/2001.07685).

* Calculate loss on weakly augmented labeled data with standatd cross-entropy loss

* Calculate loss on strongly augmented unlabeled data with standatd cross-entropy loss with pseudo-lables

* Use SimCLR transformations as strong and RandomHorizontalFlip as weak

* You can use architecture from practice

* Dataset comes as pairs `x, y`. `x` is an image from CIFAR10. `y` is a label from `[0, 10]` if datapoint is labeled and `-1` otherwise.

**Hyperparameters**

* τ = 0.7 
* λ_u = 10 (weight of unlabeled loss)
* lr = 5e-4
* num_epochs ~ 20 or more



**You will provide the following deliverables**


1. Over the course of training, record loss ber batch.
2. After each epoch calculate accuracy on test data.

In [ ]:
def q2(train_data, test_data):
    """
    train_data: An (n_train, 3, 32, 32) torchvision dataset of CIFAR10 images with values from -1 to 1

    Returns
    - a (# of training iterations, ) numpy array  losses on each iteration
    - a (# of training epochs, ) numpy array accuracies on each epoch
    """

In [ ]:
q_results(q2)

# Bonus

## The probabilistic model

*(this is a short summary of the model presented in ["Semi-supervised Learning with
Deep Generative Models"](https://arxiv.org/pdf/1406.5298.pdf))*

In the semi-supervised setting, the generative model is a little more complicated than vanilla VAE. In particular, it incorporates a new variable $y$ that represents the class of a digit $x$.

\begin{align*}
& p(x, y, z) = p(x \mid y, z) p(z) p(y) \\
& p(y) = Cat(y \mid \pi_0), \pi_0 = (1/10, \dots, 1/10) \\
& p(z) = \mathcal N(z \mid 0, I) \\
& p(x \mid y, z) = \prod_{i=1}^D p_i(y, z)^{x_i} (1 - p_i(y, z))^{1 - x_i}
\end{align*}

## The first part of the objective

Whenever we train a probabilistic model with partial observations, we interpret the unobserved variables as latent variables. Then we marginalize them. In this case, the loss function splits into two terms: one for observed variables (we denote the set of indices of observed labels $P$), another for unobserved.

\begin{equation}
L(X, y) = \sum_{i \notin P} \log p(x_i) + \sum_{i \in P} \log p(x_i, y_i)
\end{equation}

Again, we can't compute the exact values of marginal likelihoods and resort to variational lower bound on likelihood. To compute lower bounds, we define the following variational approximation:

\begin{align*}
& q(y, z \mid x) = q(y \mid x) q(z \mid y, x)\\
& \\
& q(y \mid x) = Cat(y \mid \pi(x))\\
& q(z \mid y, x) = \mathcal N(z \mid \mu_\phi(x, y), \operatorname{diag}\sigma^2_\phi(y, x))
\end{align*}

Using the variational approximation, we will obtain two lower bounds.

First, the ELBO for $\log p(x_i, y_i)$ for the observed variables (this one is similar to ELBO of VAE).

\begin{equation}
\log p(x, y) = \log \mathbb E_{p(z)} p(x, y \mid z) \geq \mathbb E_{q(z \mid y, x)} \log \frac{p(x, y \mid z) p(z)}{q(z \mid y, x)}
\end{equation}

Second, the ELBO for $\log p(x_i)$ for the unobserved variables.

\begin{equation}
\log p(x) = \log \mathbb E_{p(y)} \mathbb E_{p(z \mid y)} \log p(x\mid z, y)\geq \mathbb E_{q(y \mid x)} \mathbb E_{q(z \mid y, x)} \log \frac{p(x, y \mid z) p(z)}{q(z \mid y, x) q(y \mid x)}
\end{equation}

Finally, the joint lower bound will be

\begin{equation}
\mathcal L(X, y) = \sum_{i \in P} \mathbb E_{q(z_i \mid y_i, x_i)} \log \frac{p(x_i, y_i \mid z_i) p(z_i)}{q(z_i \mid y_i, x_i)} + \sum_{i \notin P} \mathbb E_{q(y_i \mid x_i)} \mathbb E_{q(z_i \mid y_i, x_i)} \log \frac{p(x_i, y_i \mid z_i) p(z_i)}{q(z_i \mid y_i, x_i) q(y_i \mid x_i)}
\end{equation}

We will use reparametrized Monte-Carlo estimates to approximate expectation w.r.t. $z$. To approximate expectaion w.r.t. the discrete variable $y$ we will try three different options.

In [ ]:
def b(train_data, test_data):
    """
    train_data: An (n_train, 1, 28, 28) torchvision dataset of binary MNIST images
    Returns
    - a (# of training iterations, ) numpy array  losses on each iteration
    - a (# of training epochs, ) numpy array accuracies on each epoch
    """